# Linking Dependencies

In [3]:
# imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

# Data Input

In [16]:
# read excel file
numberOfRaces = 83

raceDataFrame = pd.read_excel('data.xlsx', sheetname='Sheet1')
raceData = np.array(raceDataFrame.values)
raceData[np.argsort(raceData[:, 12])]

# init arrays
racesX = []
racesY = []
for _ in range (0, 83):
    racesX.append([])
    racesY.append([])

# separate data into X and Y, indexed by races (each as an array of horse-in-race entry)
for i in range (0, len(raceData)):
    currentRow = raceData[i]
    
    # separate data into parts
    raceNumber = int(currentRow[12])
    rowDataX = currentRow[1:12]
    rowDataY = currentRow[0]
    
    # put data into x and y arrays
    racesX[raceNumber-1].append(rowDataX)
    racesY[raceNumber-1].append(rowDataY)
    



[[array([  5.10000000e+01,   5.00000000e+01,   4.82000000e+02,
         5.10000000e+01,   5.15000000e+01,   8.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         0.00000000e+00,   1.40000000e+03]), array([  6.00000000e+00,   0.00000000e+00,   4.93000000e+02,
         5.75000000e+01,   5.75000000e+01,   4.00000000e+00,
         3.00000000e+00,   3.00000000e+00,   2.00000000e+00,
         8.00000000e-01,   1.40000000e+03]), array([  3.08000000e+02,   0.00000000e+00,   4.51000000e+02,
         5.75000000e+01,   5.75000000e+01,   3.00000000e+00,
         5.00000000e+00,   5.00000000e+00,   3.00000000e+00,
         1.30000000e+00,   1.40000000e+03]), array([  143. ,    45. ,   457. ,    51. ,    50. ,     5. ,     4. ,
           4. ,     4. ,     1.8,  1400. ]), array([   53. ,    52. ,   482. ,    57.5,    57.5,     6. ,     2. ,
           2. ,     5. ,     2.6,  1400. ]), array([   89. ,    55. ,   493. ,    58. ,    57. ,     2. ,     8. ,
           6. 

# Fit Data to GP

# Use Modified Bayesian Optimisation

## Define Kernel, Kappa, Acquisition Function, Batch Size, etc.

## Initialise GP Model with Historical Data

## Iterate for n Rounds (Races Allowed to Observe)

### Iterate through Race Profiles

### Calculate Race Learning Index

### Output Race with Max. Index

## Predict a Race and Use Decision Function to Determine Whether to Bet

## Plot Results with Different Params